Deuxième essai

# Collecte des données

In [47]:
import os
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from dotenv import load_dotenv

In [48]:
# Charger les configurations depuis un fichier JSON
def load_config(config_path):
    """
    Charge le fichier de configuration pour récupérer les informations API.
    
    Parameters:
    config_path (str): Le chemin vers le fichier de configuration.
    
    Returns:
    dict: Un dictionnaire contenant 'API_KEY' et 'BASE_URL'.
    """
    with open(config_path, 'r') as file:
        config = json.load(file)
    return config

# Charger les configurations
config = load_config('/Users/marinebaret/Desktop/ensbyum/collecte nettoyage et pretraitement/config.json')

# Variables d'environnement chargées depuis config.json
API_KEY = config.get('API_KEY')
BASE_URL = config.get('BASE_URL')

# Exemple de fonction pour récupérer les données d'articles
def collect_news_data(query, language='fr', page_size=5):
    params = {
        'q': query,
        'language': language,
        'apiKey': API_KEY,
        'pageSize': page_size
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        articles = response.json()['articles']
        data = [(article['title'], article['description']) for article in articles]
        return data
    else:
        print(f"Error: {response.status_code}")
        return None


In [55]:
def collect_news_data(query, language='fr', page_size=5):
    """
    Récupère les articles d'actualité via l'API NewsAPI.
    
    Parameters:
    query (str): Le mot-clé de recherche pour les articles.
    language (str): Langue des articles (par défaut 'fr' pour français).
    page_size (int): Nombre d'articles à récupérer (par défaut 5).
    
    Returns:
    pandas.DataFrame: Un DataFrame contenant les titres et descriptions des articles.
    """
    # Préparer les paramètres de la requête
    params = {
        'q': query,
        'language': language,
        'apiKey': API_KEY,
        'pageSize': page_size
    }
    
    # Faire la requête à l'API
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        articles = response.json()['articles']
        # Extraire les titres et descriptions
        data = [(article['title'], article['description']) for article in articles]
        # Convertir en DataFrame
        df = pd.DataFrame(data, columns=['Title', 'Description'])
        return df
    else:
        print(f"Erreur de récupération des données: {response.status_code}")
        return None



In [56]:
# Exemple d'utilisation
query = 'debates'  # Exemple de sujet
news_df = collect_news_data(query)
news_df.to_csv('news_articles.csv', index=False)

Erreur de récupération des données: 401


AttributeError: 'NoneType' object has no attribute 'to_csv'

In [58]:
print(news_df)

None


# Nettoyage des données

In [41]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


In [43]:
# Télécharger les ressources NLTK nécessaires (si ce n'est pas déjà fait)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marinebaret/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marinebaret/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
# Fonction de nettoyage des textes
def clean_text(text):
    """
    Nettoie un texte en supprimant les caractères spéciaux, les URLs, et en normalisant le texte.
    
    Parameters:
    text (str): Le texte à nettoyer.
    
    Returns:
    str: Le texte nettoyé.
    """
    # Suppression des URLs
    text = re.sub(r'http\S+', '', text)
    # Conversion en minuscules
    text = text.lower()
    # Suppression des caractères spéciaux
    text = re.sub(r'[^a-z\s]', '', text)
    return text



In [45]:
# Tokenisation et suppression des mots inutiles
def tokenize_and_remove_stopwords(text):
    """
    Tokenise un texte et supprime les mots inutiles (stopwords).
    
    Parameters:
    text (str): Le texte à tokeniser.
    
    Returns:
    list: Liste des mots filtrés.
    """
    # Tokenisation
    tokens = word_tokenize(text)
    # Liste des stopwords en anglais
    stop_words = set(stopwords.words('english'))
    # Filtrer les stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

In [46]:
# Exemple de nettoyage d'un texte
raw_text = "Here's an example of a text that contains a URL: http://example.com and some unnecessary words!"
cleaned_text = clean_text(raw_text)
tokens = tokenize_and_remove_stopwords(cleaned_text)
print(tokens)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/marinebaret/nltk_data'
    - '/Users/marinebaret/myenv/nltk_data'
    - '/Users/marinebaret/myenv/share/nltk_data'
    - '/Users/marinebaret/myenv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


Premier essai

In [34]:
# Configuration et gestion des informations sensibles
CONFIG_FILE = os.getenv("CONFIG_FILE", "config.json")

In [ ]:
# Définir le chemin complet pour le dossier de configuration
def create_config_folder():
    """Crée le dossier de configuration au chemin spécifié."""
    folder_path = "/Users/marinebaret/Desktop/ensbyum/collecte nettoyage et pretraitement"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Dossier '{folder_path}' créé avec succès.")
    else:
        print(f"Le dossier '{folder_path}' existe déjà.")

# Créer un fichier 'config.json' avec des informations par défaut dans le chemin spécifié
def create_config_file():
    """Crée un fichier config.json dans le dossier spécifié avec des données par défaut."""
    config_data = {
        "API_KEY": "your_api_key_here",
        "BASE_URL": "https://content.guardianapis.com/search"
    }
    config_path = "/Users/marinebaret/Desktop/ensbyum/collecte nettoyage et pretraitement/config.json"
    
    with open(config_path, "w") as config_file:
        json.dump(config_data, config_file, indent=4)
        print(f"Fichier '{config_path}' créé avec succès.")

if __name__ == "__main__":
    create_config_folder()
    create_config_file()


Le dossier '/Users/marinebaret/Desktop/ensbyum/collecte nettoyage et pretraitement' existe déjà.
Fichier '/Users/marinebaret/Desktop/ensbyum/collecte nettoyage et pretraitement/config.json' créé avec succès.


In [35]:
CONFIG_FILE=/Users/marinebaret/Desktop/ensbyum/collecte nettoyage et pretraitement/config.json


SyntaxError: invalid syntax (3555796477.py, line 1)

In [ ]:
# Vérifie si la variable d'environnement 'CONFIG_FILE' est définie
config_file = os.getenv("CONFIG_FILE")

if config_file:
    print(f"La variable d'environnement CONFIG_FILE est définie : {config_file}")
else:
    print("La variable d'environnement CONFIG_FILE n'est pas définie.")


La variable d'environnement CONFIG_FILE n'est pas définie.


In [33]:
# Chargement des variables d'environnement
load_dotenv()


False